In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns
import mlflow
import dagshub
import json
import os
import keras
import joblib
import matplotlib.pyplot as plt
from dagshub import dagshub_logger
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, Callback
from sklearn.metrics import confusion_matrix, classification_report
from kerastuner.tuners import BayesianOptimization

2025-05-28 21:37:52.626859: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-28 21:37:52.775989: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748461072.844401   39165 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748461072.869800   39165 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748461072.998781   39165 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
df = pd.read_csv('../data/csv/aug_landmark_dataset.csv')
df.shape

(32220, 64)

In [3]:
label_encoder = LabelEncoder()
scaler = StandardScaler()

classes = sorted(df['label'].unique())

label_encoder.fit(classes)

y = df['label']
y = label_encoder.transform(y)

X = df.drop(columns=['label'], axis=1)

# 70% train, 15% val, 15% test
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=(15/85), random_state=42, stratify=y_temp)

print(f"Partitions shape:\nTrain{X_train.shape}\nValidation: {X_val.shape}\nTest:{X_test.shape}")

X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

Partitions shape:
Train(22554, 63)
Validation: (4833, 63)
Test:(4833, 63)


In [4]:
joblib.dump(scaler, '../artifacts/csv_model/augmented/scaler.pkl')
joblib.dump(label_encoder, '../artifacts/csv_model/augmented/label_encoder.pkl')

['../artifacts/csv_model/augmented/label_encoder.pkl']

In [5]:
# Defining mlflow experiment parameter
TRIAL_NAME = "trial_"
MLFLOW_MAIN_RUN = "Main Tunining Run: Augmented Landmark Model "
TUNER_DIRECTORY = "logs/tuner"
TUNER_PROJECT_NAME = "FingerSpellIT - Augmented Landmark Model"
MODEL_NAME = "augmented_andmark_model_v1"
EVALUATION_MLFLOW_RUN = "Evaluating Augmented Landmark Model"
REPORT_NAME = "augmented_landmark_model_classification_report"
CM_NAME="augmented_landmark_model_confusion_matrix_best_model"

In [6]:
# Dagshub Initialization
mlflow.set_tracking_uri("https://dagshub.com/alfoCaiazza/FingerSpellIT.mlflow")

dagshub.init(repo_owner="alfoCaiazza", repo_name="FingerSpellIT", mlflow=True)
dagshub_log = dagshub_logger(metrics_path="metrics", hparams_path="params")

Accessing as alfoCaiazza

Initialized MLflow to track repo "alfoCaiazza/FingerSpellIT"

Repository alfoCaiazza/FingerSpellIT initialized!

In [7]:
# To avoid OOM errors, setting GPU Memory Consuption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    print(f"GPU: {gpu}")
    tf.config.experimental.set_memory_growth(gpu, True) # Keeping the use of memory limited to prevent errors

GPU: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [8]:
def build_model(hp):
    # Input Layer
    input_layer = layers.Input(shape=(63,))
    x = input_layer
    
    # First Dense Layer
    activation = hp.Choice('initial_activation', ['relu', 'tanh', 'sigmoid'])
    x = layers.Dense(64, activation=activation)(x)
    x = layers.Dropout(hp.Float('initial_dropout', 0.1, 0.5, step=0.1))(x)
    
    # Tunable Hidden Layers
    for i in range(hp.Int('num_layers', 1, 4)):
        units = hp.Int(f'units_{i}', 128, 512, step=128)
        x = layers.Dense(units, activation=activation)(x)

        # Optional BatchNorm
        if hp.Boolean(f'use_batchnorm_{i}'):
            x = layers.BatchNormalization()(x)
        x = layers.Activation(activation)(x)
            
        x = layers.Dropout(hp.Float(f'dropout_{i}', 0.1, 0.5, step=0.1))(x)
    
    # Output Layer
    prediction = layers.Dense(24, activation='softmax')(x)
    
    # Compile Model
    optimizer_name = hp.Choice('optimizer', ['adam', 'rmsprop', 'sgd'])
    
    if optimizer_name == 'adam':
        optimizer = keras.optimizers.Adam(
            learning_rate=hp.Float('adam_lr', 1e-5, 1e-2, sampling='log')
        )
    elif optimizer_name == 'rmsprop':
        optimizer = keras.optimizers.RMSprop(
            learning_rate=hp.Float('rmsprop_lr', 1e-5, 1e-2, sampling='log'),
            rho=hp.Float('rmsprop_rho', 0.8, 0.99)
        )
    elif optimizer_name == 'sgd':
        optimizer = keras.optimizers.SGD(
            learning_rate=hp.Float('sgd_lr', 1e-4, 1e-1, sampling='log'),
            momentum=hp.Float('sgd_momentum', 0.0, 0.99)
        )
    
    model = Model(inputs=input_layer, outputs=prediction)
    model.compile(
        optimizer=optimizer,
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy', 'sparse_categorical_accuracy']
    )
    
    return model

In [9]:
base_callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', patience=2, factor=0.5, min_lr=1e-5)
]

# MLflow tracking callback
class MLflowCallback(Callback):
    def __init__(self, trial_hyperparameters, trial_id):
        super().__init__()
        self.trial_id = trial_id
        self.trial_hyperparameters = trial_hyperparameters

    def on_train_begin(self, logs=None):
        self.run =  mlflow.start_run(run_name=f"{TRIAL_NAME}_{self.trial_id}", nested=True)

        mlflow.log_param('trail_id', self.trial_id)
        for param_name, param_value in self.trial_hyperparameters.values.items():
            mlflow.log_param(param_name, param_value)


    def on_epoch_end(self, epoch, logs=None):
        if logs is not None:
            for metric_name, value in logs.items():
                mlflow.log_metric(metric_name, value, step=epoch)

    def on_train_end(self, logs=None):
        if self.run:
            mlflow.end_run()

In [10]:
# Subclassed RandomSerach tuner which uses customized MLflow callback
class MLflowTuner(BayesianOptimization):
    def run_trial(self, trial, *args, **kwargs):
        callbacks = base_callbacks + [MLflowCallback(trial.hyperparameters, trial.trial_id)]
        kwargs['callbacks'] = callbacks
        return super().run_trial(trial, *args, **kwargs)

In [11]:
with mlflow.start_run(run_name=f"{MLFLOW_MAIN_RUN}"):
    epochs = 50

    mlflow.set_tag("stage", "hyperparameter_tuning")
    mlflow.set_tag("model_architecture", "ResNet50")
    mlflow.log_param('epochs', epochs)

    tuner = MLflowTuner(
        build_model,
        objective='sparse_categorical_accuracy',
        max_trials=5,
        executions_per_trial=1,
        directory=f'{TUNER_DIRECTORY}',
        project_name=f'{TUNER_PROJECT_NAME}'
    )
    
    tuner.search(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs
    )

    best_model = tuner.get_best_models(num_models=1)[0]
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
    best_trial = tuner.oracle.get_best_trials(num_trials=1)[0]

    for param, value in best_hps.values.items():
        mlflow.log_param(param, value)

    # Log metriche finali del trial
    for metric_name, metric_data in best_trial.metrics.metrics.items():
        if metric_data and isinstance(metric_data, dict):
            values = metric_data.get('value', [])
            if values:
                mlflow.log_metric(metric_name, values[-1])

    # Log best score
    mlflow.log_metric("best_sparse_categorical_accuracy", best_trial.score)

    # Salving the model
    model_path = f"../models/{MODEL_NAME}.keras"
    best_model.save(model_path)
    mlflow.log_artifact(model_path)

mlflow.end_run()

Trial 5 Complete [00h 06m 22s]
sparse_categorical_accuracy: 0.9470604062080383

Best sparse_categorical_accuracy So Far: 0.949543297290802
Total elapsed time: 00h 42m 10s


/home/acaia/FingerSpellIT/FingerSpellIT/.fingerspellit/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'SGD', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


🏃 View run Main Tunining Run: Augmented Landmark Model  at: https://dagshub.com/alfoCaiazza/FingerSpellIT.mlflow/#/experiments/0/runs/f3af42dd94c040a18d01c74425aa41d2
🧪 View experiment at: https://dagshub.com/alfoCaiazza/FingerSpellIT.mlflow/#/experiments/0


In [12]:
results = best_model.evaluate(X_test, y_test)
print("Evaluation results:", results)

y_pred_probs = best_model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)

y_true = np.array(y_test)

152/152 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9767 - loss: 0.0735 - sparse_categorical_accuracy: 0.9767
Evaluation results: [0.08032340556383133, 0.9743430614471436, 0.9743430614471436]
152/152 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step


In [13]:
# Parameters configuration
output_dir = "../artifacts/csv_model/augmented"
report_name = REPORT_NAME 
cm_name = CM_NAME          
run_name = EVALUATION_MLFLOW_RUN

os.makedirs(output_dir, exist_ok=True)

with mlflow.start_run(run_name=run_name):
    # Classification report as a JSON file
    class_names = label_encoder.classes_.tolist()
    report_json_path = os.path.join(output_dir, f"{report_name}.json")
    report = classification_report(y_true, y_pred, target_names=class_names, output_dict=True)
    print("Classification Report", json.dumps(report, indent=4))

    with open(report_json_path, "w") as f:
        json.dump(report, f, indent=4)

    # Classification report as a txt file
    report_txt_path = os.path.join(output_dir, f"{report_name}.txt")
    with open(report_txt_path, "w") as f:
        f.write(classification_report(y_true, y_pred, target_names=class_names))

    # Log on MLflow
    mlflow.log_artifact(report_json_path, "evaluation_metrics")

    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.tight_layout()

    cm_path = os.path.join(output_dir, f"{cm_name}.png")
    plt.savefig(cm_path, dpi=300, bbox_inches='tight')

    # Log dell'immagine su MLflow
    mlflow.log_figure(plt.gcf(), f"evaluation_plots/{cm_name}.png")
    plt.close()


Classification Report {
    "a": {
        "precision": 0.9945054945054945,
        "recall": 0.9945054945054945,
        "f1-score": 0.9945054945054945,
        "support": 182.0
    },
    "b": {
        "precision": 0.8410596026490066,
        "recall": 0.9694656488549618,
        "f1-score": 0.900709219858156,
        "support": 131.0
    },
    "c": {
        "precision": 0.9625668449197861,
        "recall": 0.994475138121547,
        "f1-score": 0.9782608695652174,
        "support": 181.0
    },
    "d": {
        "precision": 0.9746835443037974,
        "recall": 0.9390243902439024,
        "f1-score": 0.9565217391304348,
        "support": 164.0
    },
    "e": {
        "precision": 0.9953703703703703,
        "recall": 0.9907834101382489,
        "f1-score": 0.9930715935334873,
        "support": 217.0
    },
    "f": {
        "precision": 0.9382716049382716,
        "recall": 0.8587570621468926,
        "f1-score": 0.8967551622418879,
        "support": 177.0
    },
    "g